In [97]:
import pandas as pd
import re
 

column_names1 = ['userId','movieId','rating','timestamp'] 
ratings_dataset = pd.read_csv("C:/Users/DELL/Documents/second semester in 3'rd year/NLP/Movie recommender/ratings.csv",
                            sep=';',header=None,names=column_names1) 
ratings_dataset = ratings_dataset.tail(-1)
ratings_dataset = ratings_dataset.drop('timestamp' , axis = 1)
ratings_dataset=ratings_dataset.loc[0:100000]
ratings_dataset

C:\Users\DELL\AppData\Local\Temp\ipykernel_12516\3395150508.py:6: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_dataset = pd.read_csv("C:/Users/DELL/Documents/second semester in 3'rd year/NLP/Movie recommender/ratings.csv",


,userId,movieId,rating
1,1,1193,5
2,1,661,3
3,1,914,3
4,1,3408,4
5,1,2355,5
...,...,...,...
99996,669,585,4
99997,669,586,3
99998,669,587,3
99999,669,588,4


In [98]:
user_movie = pd.pivot_table(ratings_dataset, index = ['userId'], values='movieId', aggfunc='count')
user_movie[user_movie.movieId>100]
user_id = user_movie[user_movie.movieId>0].index


movie_rated = pd.pivot_table(ratings_dataset, index=['movieId'], values='userId', aggfunc='count')
movie_rated[movie_rated.userId>100]
movie_id = movie_rated[movie_rated.userId>0].index


filter_data = ratings_dataset[(ratings_dataset['userId'].isin(user_id))&(ratings_dataset['movieId'].isin(movie_id))]

filter_data


,userId,movieId,rating
1,1,1193,5
2,1,661,3
3,1,914,3
4,1,3408,4
5,1,2355,5
...,...,...,...
99996,669,585,4
99997,669,586,3
99998,669,587,3
99999,669,588,4


In [99]:
from surprise import accuracy
from surprise import SVD , Dataset, Reader
from sklearn.decomposition import TruncatedSVD
from numpy import array
from surprise.model_selection import KFold
from sklearn.model_selection import train_test_split

from surprise.model_selection import cross_validate


reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(filter_data[['userId','movieId','rating']], reader)

train_set = data.build_full_trainset()
test_set = train_set.build_anti_testset()
#kf = KFold(n_splits=3000,shuffle=True,random_state=100)

#train, test = train_test_split(filter_data, test_size=0.3)

#data_train = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)
#train_set = data_train.build_full_trainset()


#data_test = Dataset.load_from_df(test[['userId', 'movieId', 'rating']], reader)
#test_set=train_set.build_anti_testset()

svd = SVD()
svd.fit(train_set)

 

predictions = svd.test(test_set)

 

print( 'Root mean squre error :' + str(accuracy.rmse(predictions)))
"""
for trainset , testset in kf.split(data):

    svd.fit(trainset)

    predictions = svd.test(testset)

    print(accuracy.rmse(predictions))
"""

RMSE: 0.5685
Root mean squre error :0.5684748742872109


'\nfor trainset , testset in kf.split(data):\n\n    svd.fit(trainset)\n\n    predictions = svd.test(testset)\n\n    print(accuracy.rmse(predictions))\n'

In [100]:
from collections import defaultdict
def get_top_n(predictions, n):
    
    top_movies = defaultdict(list)
    for user, movie, actual_rate, estimate_rate , _ in predictions:
        top_movies[user].append((movie, estimate_rate))
        
    
    for user, user_ratings in top_movies.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_movies[user] = user_ratings[:n]
    
    return top_movies

In [101]:
top_movies = get_top_n(predictions, 10)

In [102]:
import pandas as pd
import re
 
with open("C:/Users/DELL/Downloads/movies.csv",  encoding="utf8", errors='ignore') as f:
    contents = f.read() 
lines = contents.split("\n")
 
pattern = re.compile(r'^(\d+);(.*?);(.*)$', re.MULTILINE)
 
genersList = ['Action', 'Adventure', 'Animation', "Children's",
          'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
          'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
          'Sci-Fi', 'Thriller', 'War', 'Western']
 
data =  pd.DataFrame()
movie = []
title1 = []
genres1 = []
pattern = re.compile(r'^(\d+);(.*?);(.*);$', re.MULTILINE)
for line in lines:
    for match in pattern.finditer(line):
        movieId = match.group(1)
        title = match.group(2)
       
        if any(item in match.group(3) for item in genersList):
            genres = match.group(3)
            genres1.append(genres)
        else:
            title = title + ','  + match.group(3)
            genres1.append('')
           
        title1.append(title)
        movie.append(movieId)
        
data['movieId'] = movie
data['title'] = title1
data['genres'] = genres1   

data

    


,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3877,3948,Meet the Parents (2000),Comedy
3878,3949,Requiem for a Dream (2000),Drama
3879,3950,Tigerland (2000),Drama
3880,3951,Two Family House (2000),Drama


In [103]:
# extract features from 'genres' column
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
features = vectorizer.fit_transform(data['genres'])
#print(vectorizer.get_feature_names_out())
print(features.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [104]:
from nltk.stem import WordNetLemmatizer 
import nltk
# initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# define function to lemmatize a string
def lemmatize_text(text):
    words = nltk.word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# apply lemmatization to 'title' column
data['title_lemmatized'] = data['title'].apply(lemmatize_text)
data['title_lemmatized']

0                         Toy Story ( 1995 )
1                           Jumanji ( 1995 )
2                  Grumpier Old Men ( 1995 )
3                 Waiting to Exhale ( 1995 )
4       Father of the Bride Part II ( 1995 )
                        ...                 
3877               Meet the Parents ( 2000 )
3878            Requiem for a Dream ( 2000 )
3879                      Tigerland ( 2000 )
3880               Two Family House ( 2000 )
3881                Contender , The ( 2000 )
Name: title_lemmatized, Length: 3882, dtype: object

In [105]:
# remove year from 'title' column
data['title'] = data['title'].str.replace(r'\s*\(\d{4}\)', '')
data['title']


C:\Users\DELL\AppData\Local\Temp\ipykernel_12516\934756857.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['title'] = data['title'].str.replace(r'\s*\(\d{4}\)', '')


0                         Toy Story
1                           Jumanji
2                  Grumpier Old Men
3                 Waiting to Exhale
4       Father of the Bride Part II
                   ...             
3877               Meet the Parents
3878            Requiem for a Dream
3879                      Tigerland
3880               Two Family House
3881                 Contender, The
Name: title, Length: 3882, dtype: object

In [106]:
#check duplicate rows
DuplicateNum1=data.duplicated().sum()
print(DuplicateNum1)
DuplicateNum2=ratings_dataset.duplicated().sum()
print(DuplicateNum2)

# if exist any duplicates

# drop duplicate rows
data = data.drop_duplicates()
ratings_dataset = ratings_dataset.drop_duplicates()

0
0


In [107]:
#check nulls
NullNum1=data.isnull().sum()
print(NullNum1)
NullNum2=ratings_dataset.isnull().sum()
print(NullNum2)

# if exist null values

# remove nulls from data
data = data.dropna()
ratings_dataset = ratings_dataset.dropna()

movieId             0
title               0
genres              0
title_lemmatized    0
dtype: int64
userId     0
movieId    0
rating     0
dtype: int64


In [108]:
data = data.drop('genres' , axis = 1)

data

,movieId,title,title_lemmatized
0,1,Toy Story,Toy Story ( 1995 )
1,2,Jumanji,Jumanji ( 1995 )
2,3,Grumpier Old Men,Grumpier Old Men ( 1995 )
3,4,Waiting to Exhale,Waiting to Exhale ( 1995 )
4,5,Father of the Bride Part II,Father of the Bride Part II ( 1995 )
...,...,...,...
3877,3948,Meet the Parents,Meet the Parents ( 2000 )
3878,3949,Requiem for a Dream,Requiem for a Dream ( 2000 )
3879,3950,Tigerland,Tigerland ( 2000 )
3880,3951,Two Family House,Two Family House ( 2000 )


In [109]:
user_id=input("Enter User ID : ")
userID = [str(user_id)]

for user, user_ratings in top_movies.items():
    if user in userID:
        for (movieID, rating) in user_ratings:
            x=data.loc[data['movieId'] == str(movieID)]
            f=list(x['title'])
            movie=f[0]
            print('Movie:', movieID, '-', str(movie), ',Predicted Rating:', str(rating))

Enter User ID : 1
Movie: 318 - Shawshank Redemption, The ,Predicted Rating: 4.807492096585229
Movie: 1172 - Cinema Paradiso ,Predicted Rating: 4.788381530033483
Movie: 1950 - In the Heat of the Night ,Predicted Rating: 4.772519064553059
Movie: 858 - Godfather, The ,Predicted Rating: 4.767082677437646
Movie: 2019 - Seven Samurai (The Magnificent Seven) (Shichinin no samurai) ,Predicted Rating: 4.7471868440092155
Movie: 750 - Dr. Strangelove or, How I Learned to Stop Worrying and Love the Bomb ,Predicted Rating: 4.73836965969689
Movie: 1262 - Great Escape, The ,Predicted Rating: 4.728838977002609
Movie: 593 - Silence of the Lambs, The ,Predicted Rating: 4.727626575925375
Movie: 2329 - American History X ,Predicted Rating: 4.7272507980170655
Movie: 899 - Singin' in the Rain ,Predicted Rating: 4.718769973455278


In [110]:
import pandas as pd
import re
column_names1 = ['userId','gender','age','occupation','zip-code'] 
users_dataset = pd.read_csv("C:/Users/DELL/Documents/second semester in 3'rd year/NLP/Movie recommender/users.csv"
                            ,sep=';',header=None,names=column_names1) 
users_dataset = users_dataset.tail(-1)
users_dataset

,userId,gender,age,occupation,zip-code
1,1,F,1,10,48067
2,2,M,56,16,70072
3,3,M,25,15,55117
4,4,M,45,7,2460
5,5,M,25,20,55455
...,...,...,...,...,...
6036,6036,F,25,15,32603
6037,6037,F,45,1,76006
6038,6038,F,56,1,14706
6039,6039,F,45,0,1060
